In [2]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pymongo
from config import mongoConn

In [3]:
# URL of page to be scraped
query = 'Blazing Saddles'
base_url = 'https://reelgood.com'
url = f'{base_url}/search?q={query}'
print(url)

https://reelgood.com/search?q=Blazing Saddles


In [4]:
# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')
link = soup.select('.e1qyeclq5 a')[0]['href']
show_url = base_url + link
print(show_url)

https://reelgood.com/movie/blazing-saddles-1974


In [5]:
show_response = requests.get(show_url)
show_soup = BeautifulSoup(show_response.text, 'html.parser') 

In [6]:
#services = show_soup.select('.e126mwsw1 span[class*=hou11]')
services = show_soup.select('.e126mwsw1 span[class*=hou113]') 

In [7]:
title = ''
desc = '' 
feature_img = '' 
meta = ''
recommended = ''
try:
    title = show_soup.select('h1.e14injhv7')[0].text
    desc = show_soup.select('p[itemprop=description]')[0].text 
    feature_img = show_soup.select('.e1x40mdt0 picture.e1181ybh0 img.e1181ybh1')[0]['src']  
    meta = show_soup.select('.ey4ir3j2') 
    recommended = show_soup.select('.e1yfir8f4 .e1qyeclq4') 
except:
    print('error') 

In [8]:
dic_list = {
    'query': query,
    'services': [s.text for s in set(services)],
    'title': title,
    'description': desc,
    'feature_img': feature_img ,
    'service_url': '',
    'info': '',
    'recommended':[r.text for r in recommended],
    'meta': ''
} 
#dic_list

In [9]:
#conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(mongoConn) 
db = client.shows_db
collection = db.items
collection.insert_one(dic_list)

In [11]:
results = collection.find({}, {'_id': False})
coll_df = pd.DataFrame(results)
coll_json = coll_df.to_json(orient='records') 
coll_json

'[{"query":"Amelie","services":["CBS All Access","Rent","PlutoTV","Buy"],"title":"Am\\u00e9lie","description":"At a tiny Parisian caf\\u00e9, the adorable yet painfully shy Am\\u00e9lie accidentally discovers a gift for helping others. Soon Amelie is spending her days as a matchmaker, guardian angel, and all-around do-gooder. But when she bumps into a handsome stranger, will she find the courage to become the star of her very own love story?Am\\u00e9lie featuring Audrey Tautou and Mathieu Kassovitz is streaming on CBS All Access, free on PlutoTV, available for rent or purchase on iTunes, and 1 other. It\'s a comedy and romance movie with a high IMDb audience rating of 8.3 (703,668 votes) and was very well received by critics.","feature_img":"https:\\/\\/img.reelgood.com\\/content\\/movie\\/1c891396-74d8-4aa6-a25b-25dfdda9466a\\/poster-780.jpg","service_url":"","info":"","recommended":["Sleepy Hollow (1999)","Alien Resurrection (1997)","Delicatessen (1991)","The Da Vinci Code (2006)","T